In [163]:
#library to extract subtitles
#%pip install youtube_transcript_api

^C
Traceback (most recent call last):
  File "/Users/Rolf/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Rolf/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Rolf/anaconda3/lib/python3.7/site-packages/pip/__main__.py", line 23, in <module>
    from pip._internal.cli.main import main as _main  # isort:skip # noqa
  File "/Users/Rolf/anaconda3/lib/python3.7/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/Users/Rolf/anaconda3/lib/python3.7/site-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/Users/Rolf/anaconda3/lib/python3.7/site-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "/Users/Rolf/anaconda3/lib/python3.7/site-packages/pip/_i

In [55]:
#code needed to use colab
#if you're using juptyer notebook then skip this line
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [56]:
#cd used in colab
#%cd "gdrive/My Drive/Colab Notebooks"
%cd ~/Documents/GitHub/esade_fake_news/1_Climate_Change/youtube_query/data/nlp
!ls

/Users/Pierre/Documents/GitHub/esade_fake_news/1_Climate_Change/youtube_query/data/nlp
00000000-000000_climate_change_bias.csv
00000000-000000_climate_change_nlp.csv
00000000-000000_climate_change_nlp_subs.csv
00000000-000000_climate_change_nlp_subs_clean.csv
00000000-000000_climate_change_nlp_subs_clean_expanded.csv
20200507-033205_climate_change_bias.csv
20200507-033205_climate_change_nlp.csv
20200507-033205_climate_change_nlp_subs.csv
20200507-033205_climate_change_nlp_subs2.csv
20200507-033205_climate_change_nlp_subs_clean.csv
20200507-033205_climate_change_nlp_subs_clean_expanded.csv
notebooks


In [57]:
import pandas as pd
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
from tqdm import tqdm

# Extract Subtitles

In [58]:
#import csv
df = pd.read_csv('00000000-000000_climate_change_nlp.csv', index_col=0)
df.head(2)

,Bias_num,title,description,channel,id
1,0.0,Watch the Latest News Headlines and Live Event...,#coronavirus #covid19 #news #LiveNews #Streami...,ABC News,w_Ma8oQLmSM
2,0.0,An Inconvenient Border: Where China Meets Nort...,Bob Woodruff’s daring 880-mile journey along t...,ABC News,8wRLQ8MVi0Q


In [59]:
#length of dataset
print(f'Length of dataset is: {len(df)}')

#percentage of videos that are right
pr_right = round(df['Bias_num'].mean()*100,2)
print(f'Percentage of videos that are right: {pr_right} %')

Length of dataset is: 664
Percentage of videos that are right: 18.54 %


In [60]:
#loop to extract subtitles

tttt = []

with tqdm(total=len(df['id'])) as pbar:  
    for idx in df['id']:
        try:
            t = YouTubeTranscriptApi.get_transcript(idx)
        except:
            tttt.append([idx, np.nan]) #add NaN to videos that do not have subtitles
            continue
        tt = [i["text"] for i in t]
        ttt = [' '.join(tt)]
        tttt.append([idx, ttt])
        pbar.update(1)

df_sub = pd.DataFrame(tttt, columns=['video_id', 'subtitles'])
df_sub.head()

 84%|████████▍ | 559/664 [08:42<01:38,  1.07it/s]


,video_id,subtitles
0,w_Ma8oQLmSM,NaN
1,8wRLQ8MVi0Q,[[Music] yeah Irish aqua five bucks North Nort...
2,TO9d16c2XRM,[you can't say it but you know it's true good ...
3,38DY30I73lg,[[Applause] [Music] there is a certainty to li...
4,6OQAHcB72dg,[honestly what they say doesn't bother me I un...


In [61]:
#merge subtitles with dataframe containing video id, channel, etc. 
df2 = df.merge(df_sub, left_on='id', right_on='video_id')

In [62]:
#i checked a few nan subtitles and they don't have subtitles. 
df2[df2['subtitles'].isna()].head()

,Bias_num,title,description,channel,id,video_id,subtitles
0,0.0,Watch the Latest News Headlines and Live Event...,#coronavirus #covid19 #news #LiveNews #Streami...,ABC News,w_Ma8oQLmSM,w_Ma8oQLmSM,NaN
1,0.0,Watch the Latest News Headlines and Live Event...,#coronavirus #covid19 #news #LiveNews #Streami...,ABC News,w_Ma8oQLmSM,w_Ma8oQLmSM,NaN
2,0.0,Watch LIVE: World Health Organization update o...,#coronavirus #covid19 #news #LiveNews #Streami...,ABC News,w_Ma8oQLmSM,w_Ma8oQLmSM,NaN
3,0.0,Watch LIVE: World Health Organization update o...,#coronavirus #covid19 #news #LiveNews #Streami...,ABC News,w_Ma8oQLmSM,w_Ma8oQLmSM,NaN
8,0.0,Trapped in the volcano: How the cruise of a li...,"In December 2019, the Ovation of the Seas left...",ABC News In-depth,fndcCyYzXKg,fndcCyYzXKg,NaN


In [63]:
#drop column
df2.drop('video_id', inplace=True, axis=1)

In [82]:
df2.shape

(722, 6)

In [64]:
#export csv
df2.to_csv('00000000-000000_climate_change_nlp_subs.csv')

# Subtitles cleaning

In [84]:
# load train_cleaned.csv
filename = '00000000-000000_climate_change_nlp_subs.csv'
df = pd.read_csv(filename, usecols=['Bias_num', 'subtitles','channel'])

#copy to not destroy original dataset
df_channel = df.copy()

#create new column text. easier in the analysis
df['text'] = df['subtitles']

#drop subtitles and channel
df.drop(['subtitles'], axis=1, inplace=True)

In [88]:
#which videos have NaN as subtitle
df_nan = df[df['text'].isna()]

#number of videos with no subtitles
num_nan = len(df_nan)
print(f'Number of videos that do NOT have subtitles: {num_nan}')

print()

#displays channels with missing subtitles
channel = df_nan.groupby(["channel"])['Bias_num'].count().sort_values(ascending=False)
print(channel[:5])

print()

#display distribution of bias with missing subtitles
bias = df_nan.groupby('Bias_num')['channel'].count()
print(bias)

print(1-(num_nan/len(df)))

Number of videos that do NOT have subtitles: 163

channel
TEDx Talks              21
Yahoo Finance           10
PBS NewsHour             9
Wendover Productions     8
CNN                      8
Name: Bias_num, dtype: int64

Bias_num
0.0    112
1.0     13
Name: channel, dtype: int64
0.7742382271468145


In [67]:
#drop channel
df.drop(['channel'], axis=1, inplace=True)

#drop NaNs
df.dropna(inplace=True)

#remove brackets [] and quotations marks
df['text'] = df['text'].str[2:-2]

df.head(2)

,Bias_num,text
4,0.0,[Music] yeah Irish aqua five bucks North North...
5,0.0,you can't say it but you know it's true good e...


In [68]:
#average right wing videos
df['Bias_num'].mean()

0.19331742243436753

In [69]:
#split channel string to remove each element from subtitles

#remove (in English) since the brackets cause problems with replacing values
df_channel['channel'] = df_channel['channel'].replace("(in English)", '', regex=True)

#list all unique channels
unique_channel = df_channel['channel'].unique()

#split channel name by whitespace
unique_channel_split = [i.split(' ') for i in unique_channel]

#display first row
unique_channel_split[0]

['ABC', 'News']

In [70]:
#loop to remove each single word in channel name
#doing this to remove any possibility that the NLP model learns channel to predict bias
for channel in unique_channel_split:
    for element in channel:
        df['text'].replace(element,
                           '',
                           inplace=True,
                           regex=True)

In [71]:
#the text [Music] appears in subtitles when music is playing. 
#these values mess up the analysis in the end with 3-gram, 2-gram
to_replace = ['Music', 'music', 'Applause', 'applause', 'Laughter', 'laughter']

df['text'].replace(to_replace, '', inplace=True, regex=True)  

In [80]:
df

,Bias_num,text
4,0.0,[] yeah Irish aqua five bucks North North Kore...
5,0.0,you can't say it but you know it's true good e...
6,0.0,[] [] there is a certaty to life Roberts Coun...
7,0.0,honestly what they say doesn't bother me I und...
12,0.0,do I believe UFOs or extraterrestrial visitor...
...,...,...
714,0.0,(reverent ) [Narrator] It was\na road less tr...
715,0.0,[Woman] Your support helps\nus brg you progra...
716,0.0,a busess empire needs decisive leadership but ...
718,0.0,Johns Hopks on the most respected stitutions c...


In [72]:
df.to_csv('00000000-000000_climate_change_nlp_subs_clean.csv')

## Split subtitles after 1'000 characters

We split at 1'000 characters to make our dataset larger. I (Rolf) thinks that having more rows is more important for the model training than few but long rows. I think that 1'000 characters is a good cut-off. This is an arbitrary number. 

In [73]:
# load train_cleaned.csv
filename = '00000000-000000_climate_change_nlp_subs_clean.csv'
df_clean = pd.read_csv(filename, index_col=0)

df_clean.head()

,Bias_num,text
4,0.0,[] yeah Irish aqua five bucks North North Kore...
5,0.0,you can't say it but you know it's true good e...
6,0.0,[] [] there is a certaty to life Roberts Coun...
7,0.0,honestly what they say doesn't bother me I und...
12,0.0,do I believe UFOs or extraterrestrial visitor...


In [74]:
#df_clean has this many rows

print('df_clean has this many rows: ' + str(len(df_clean)))

df_clean has this many rows: 419


In [75]:
#function to split text into columns after 1'000 characters
def chunks(s, n):
    """Produce `n`-character chunks from `s`."""
    for start in range(0, len(s), n):
        yield s[start:start+n]

In [76]:
#empty lists to keep info
sub = []
bias = []

#loop to split at 1'000 characters
for index, row in df.iterrows():
    for chunk in chunks(row['text'], 1000):
        sub.append(chunk)
        bias.append(row['Bias_num'])

In [77]:
#create a dataframe from sub and bias list
df_sub = pd.DataFrame(list(zip(bias, sub)), columns=['bias_num', 'text'])

length = len(df_sub)
print(f"Number of rows after splitting at 1'000 characters: {length}")

percentage = round(df_sub['bias_num'].mean()*100, 2)
print(f"Percentage of rows that are right wing: {percentage}%")

df_sub.head()

Number of rows after splitting at 1'000 characters: 9970
Percentage of rows that are right wing: 28.08%


,bias_num,text
0,0.0,[] yeah Irish aqua five bucks North North Kore...
1,0.0,eader it's been downhill sce despite North Ko...
2,0.0,ndos connectg to the North Korean city cn a J...
3,0.0,clusive neighbors from across the Yalu all alo...
4,0.0,it's an unclear le the middle the river but...


In [78]:
#loop to calculate average length of text per row

length = []

for index, row in df_sub.iterrows():
    length.append(len(row['text']))
    
avg_length = round(sum(length)/len(length),2)
print(f"Average character length of text column per row: {avg_length}")

Average character length of text column per row: 979.78


In [79]:
df_sub.to_csv('00000000-000000_climate_change_nlp_subs_clean_expanded.csv')